In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [22]:
start_date = '20220101'
end_date = '20230131'

In [23]:
query = f"""

with upi_base as (
select 
    DATE_FORMAT(DATE_TRUNC('month', CAST(date_parse(yyyymmdd, '%Y%m%d') AS DATE)), '%Y-%m-%d') time_level,
    yyyymmdd,status, 
    user_id captain_id, id, 
    approver,amount,upidetails_upi_id upi_id
from payments.redeem_transactions_snapshot
where 
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and status in ('Credited')--, 'rejected')
    -- and upidetails_upi_id = 'rapido1122@sbi'
    and upidetails_upi_id in ('yuvadeepu14@okicici', 'rapido57@ybl', 'rapido123@kotak', 'rapido12@dbs', 'rapido1122@sbi', 'ola1122@ibl', 'nnnsales9999@ybl', 'nnnsales9991@ybl', 'nnnsales999@icici', 'nagesha.m1@ybl', 'arya1929@icici', 'anandish@fbl', '9620685@ybl', '8431720039@jupiteraxis', '70904222559@ybl', '7090422255@icici' )
),

order_logs_snapshot as (
select 
    DATE_FORMAT(DATE_TRUNC('month', CAST(date_parse(yyyymmdd, '%Y%m%d') AS DATE)), '%Y-%m-%d') time_level,
    service_obj_service_name service, 
    order_id, captain_id, captain_obj_device_id, accept_to_pickup_distance a2p, 
    spd_fraud_flag, ride_time, distance_final_distance
from orders.order_logs_snapshot 

where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and service_obj_service_name in ('Auto', 'Link')
    and order_status = 'dropped' 
    and captain_id in (select distinct captain_id from upi_base)
   
    -- and captain_obj_device_id != '00000000'
    -- and captain_obj_device_id is not null
    -- and captain_obj_device_id != ''
),


view as (
select 
    upi.time_level,
    upi.upi_id,
    o.service,
    upi.captain_id,
    o.captain_obj_device_id,
    o.order_id,
    o.spd_fraud_flag, 
    o.distance_final_distance,
    o.ride_time,
    a2p
    
from upi_base upi
left join order_logs_snapshot o on upi.captain_id = o.captain_id --and upi.time_level = o.time_level
where o.service is not null
)

select  
    -- time_level,
    upi_id,
    --service,
    count(distinct captain_id) total_captains,
    count(distinct captain_obj_device_id) total_devices,
    count(distinct order_id) total_orders,
    count(distinct case when (spd_fraud_flag = true or ride_time <= 1 or distance_final_distance*1.00/(ride_time*1.00/60) >= 250) then order_id end) total_spd_orders,
    count(distinct case when a2p = 0 then order_id end ) a2p_zero,
    try(count(distinct case when a2p = 0 then order_id end )*100.00/count(distinct order_id) ) a2p_zero_perc,
    count(distinct case when a2p < 0.05 then order_id end ) a2p_50,
    try(count(distinct case when a2p < 0.05 then order_id end )*100.00/count(distinct order_id) ) a2p_50_perc
    
from view
group by 1
order by 1

"""

In [24]:
df_jan22_jan23 = pd.read_sql(query, conn)
df_jan22_jan23

,upi_id,total_captains,total_devices,total_orders,total_spd_orders,a2p_zero,a2p_zero_perc,a2p_50,a2p_50_perc
0,70904222559@ybl,27,32,2885,19,2714,94.07,2804,97.19
1,7090422255@icici,221,226,48651,259,46000,94.55,47347,97.32
2,8431720039@jupiteraxis,111,110,37679,28,31972,84.85,32138,85.29
3,9620685@ybl,189,63,6658,132,4649,69.83,5167,77.61
4,anandish@fbl,256,49,6690,143,5739,85.78,6153,91.97
5,arya1929@icici,67,146,8891,14,8383,94.29,8686,97.69
6,nagesha.m1@ybl,47,55,3015,3,2994,99.30,3005,99.67
7,nnnsales9991@ybl,197,118,23598,54,23147,98.09,23382,99.08
8,nnnsales9999@ybl,154,108,19306,31,19047,98.66,19184,99.37
9,nnnsales999@icici,17,33,2335,4,2101,89.98,2209,94.60


In [9]:
df_dec_jan = pd.read_sql(query, conn)
df_dec_jan

,upi_id,service,total_captains,total_devices,total_orders,total_spd_orders,a2p_zero,a2p_zero_perc,a2p_50,a2p_50_perc
0,rapido1122@sbi,Auto,266,140,37599,40,31361,83.41,31559,83.94
